# 使用TimesFM作为特征提取器

本笔记本演示了如何使用 `timesfm.TimesFM_2p5_200M_torch()` 预训练模型从时间序列数据中提取特征。

In [9]:
import torch
import timesfm
import numpy as np
import pandas as pd
import os
import sys

# Add project root to Python path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## 1. 加载TimesFM模型

In [10]:
# 加载TimesFM模型
tfm = timesfm.TimesFM_2p5_200M_torch()
tfm.load_checkpoint()
tfm.compile(
    timesfm.ForecastConfig(
        max_context=1024,
        max_horizon=256,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )
)

/home/userroot/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 19152.07it/s]



## 2. 准备示例数据

为了演示特征提取，我们创建一些合成的时间序列数据。TimesFM期望的输入格式是一个包含多个`numpy`数组的列表，其中每个数组代表一个时间序列。

In [11]:
# 准备示例数据
print("创建测试数据...")

# 创建测试时间序列数据
test_inputs = [
    np.linspace(0, 1, 100),
    np.sin(np.linspace(0, 20, 67)),
]

print(f"输入数据包含 {len(test_inputs)} 个时间序列")
print(f"第一个序列长度: {len(test_inputs[0])}")
print(f"第二个序列长度: {len(test_inputs[1])}")

创建测试数据...
输入数据包含 2 个时间序列
第一个序列长度: 100
第二个序列长度: 67


## 3. 使用TimesFM进行预测（特征提取）

使用TimesFM模型对时间序列进行预测，这也可以看作是从时间序列中提取特征的过程。

In [12]:
# 使用TimesFM进行预测
print("使用TimesFM进行预测...")
point_forecast, quantile_forecast = tfm.forecast(
    horizon=12,
    inputs=test_inputs
)

使用TimesFM进行预测...


In [13]:
# 查看预测结果
print(f"点预测结果形状: {point_forecast.shape}")  # (2, 12)
print(f"分位数预测结果形状: {quantile_forecast.shape}")  # (2, 12, 10): mean, then 10th to 90th quantiles.

print("\n第一个序列的点预测结果:")
print(point_forecast[0])

print("\n第二个序列的点预测结果:")
print(point_forecast[1])

点预测结果形状: (2, 12)
分位数预测结果形状: (2, 12, 10)

第一个序列的点预测结果:
[1.0127585 1.0188391 1.0272739 1.0430081 1.0479407 1.0591323 1.074695
 1.0754818 1.0890968 1.1026995 1.1116321 1.1225954]

第二个序列的点预测结果:
[ 0.9855348   0.97798806  0.89091337  0.7285633   0.47109857  0.19757876
 -0.10510507 -0.39542463 -0.631007   -0.8359534  -0.96646345 -1.0037616 ]


## 4. 特征提取说明

TimesFM模型的预测结果可以作为时间序列的特征表示。特别是分位数预测结果包含了丰富的统计信息，可以用于下游任务。

- 点预测结果形状为 `(n_series, horizon)`
- 分位数预测结果形状为 `(n_series, horizon, n_quantiles)`，其中包含均值和10个分位数(10th到90th)

这些预测结果可以被展平并用作机器学习模型的输入特征。

In [14]:
# 将预测结果转换为特征向量
# 合并点预测和分位数预测作为特征
features = np.concatenate([point_forecast[:, :, np.newaxis], quantile_forecast], axis=2)

print(f"第一个序列的特征向量长度: {features[0].size} ({features.shape[1]}*{features.shape[2]})")
print(f"第二个序列的特征向量长度: {features[1].size} ({features.shape[1]}*{features.shape[2]})")

# 展平为一维特征向量
flattened_features = features.reshape(features.shape[0], -1)
print(f"\n展平后的特征形状: {flattened_features.shape}")

第一个序列的特征向量长度: 132 (12*11)
第二个序列的特征向量长度: 132 (12*11)

展平后的特征形状: (2, 132)
